<a href="https://colab.research.google.com/github/compartia/nlp_tools/blob/in-tensors/FuzzyMatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
GLOBALS__={}

## MAIN, init, load code

In [0]:
#@title  { form-width: "300px", display-mode: "form" }
import os






# ''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''
import sys
# ====================================
# ====================================
_git_branch = "in-tensors"  # @param {type:"string"}
# ====================================
# ====================================


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

  # self-test
  from text_tools import untokenize
  print(untokenize(['code', 'imported', 'OK 👍']))

  print('installing antiword...')
  exec('sudo apt-get install antiword')

  print('installing docx2txt...')
  exec("pip install docx2txt")

  GLOBALS__['_init_import_code_from_gh'] = True

  ''' AZ:-------------------------------------------------IMPORT CODE GITHUB-😺---'''
  print('❤️ DONE importing Code fro GitHub')


# AZ:-INIT ELMO-----------------------------------------------------------------------------------

import tensorflow_hub as hub

#
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML


 

# AZ:- THE CODE----------------------------------------------------------------------------------

def _init_the_code(reset=False):
  if '_init_the_code' in GLOBALS__ and not reset:
    print('👌 Code is alredy imported!')
    return

  from renderer import HtmlRenderer
  from renderer import to_multicolor_text
  from renderer import known_subjects_dict

  from structures import ContractSubject
  from contract_parser import ContractDocument3

  from ml_tools import ProbableValue

  from legal_docs import LegalDocument
  from renderer import as_warning, as_headline_3, as_offset, as_smaller

  class DemoRenderer(HtmlRenderer):

    def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
      display(HTML(html))

    def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

    def render_multicolor_text(self, tokens, vectors, colormap, min_color=None, _slice=None):
      display(HTML(to_multicolor_text(tokens, vectors, colormap, min_color=min_color, _slice=_slice)))

     
     

     
  GLOBALS__['renderer'] = DemoRenderer()

  GLOBALS__['_init_the_code'] = True
  print("❤️ DONE initializing the code")

  # AZ:-------------------------------------------------Init Protocols context===


def read_doc(fn):
  import docx2txt, sys, os

  text = ''
  try:
    text = docx2txt.process(fn)

  except:
    print("Unexpected error:", sys.exc_info())
    os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
    with open(fn + '.txt') as f:
      text = f.read()

  return text

def interactive_upload(filetype):
  from google.colab import files
  import docx2txt

  print(f'Please select "{filetype}" .docx file:')
  uploaded = files.upload()
  docs = []
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

    with open(fn, "wb") as df:
      df.write(uploaded[fn])
      df.close()

    # extract text

    text = ''
    try:
      text = docx2txt.process(fn)
    except:
      print("Unexpected error:", sys.exc_info())
      os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
      with open(fn + '.txt') as f:
        text = f.read()
    print("Символов в оригинальном документе:", len(text))
    docs.append(text)
    return docs

  
# AZ:- ENDO OF THE THE CODE------------------------------------------------XXXX


# 1.
_init_import_code_from_gh()
 
# 3.
_init_the_code(True)



### the model 💎 for finding closest embedding to a pattern
1. At first stage, we calculate attention vectors (AV) for every pattern
2. Then, for each AV we're finding the closest point (`best point`) in text embedding space, 
3. Then we're calculating "improved" vectors -- we calculate distance from the `"best point"` (actually, from the best *window*)  to each point in text embedding space.  
  
WARNING: this should be used only if `best point` is close enough to the pattern

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from tf_model import PatternSearchModel
PM = PatternSearchModel(tf, hub)


## Playground

In [0]:
 
want_upload = True #@param {type:"boolean"}
search_for = "\u0432 \u043B\u0438\u0446\u0435 \u0433\u0435\u043D\u0435\u0440\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u0434\u0438\u0440\u0435\u043A\u0442\u043E\u0440\u0430" #@param {type:"string"}
from patterns import AbstractPatternFactory, FuzzyPattern

from text_tools import tokenize_text
import re
from text_normalize import normalize_text, replacements_regex
uploaded = """
Берлинская дада-ярмарка и Международная выставка сюрреализма в париж­ской галерее «Изящные искусства» в 1938 году стали высшими точками развития двух движений и подвели им итог. На «Сюрреалис­тической улице», за манекенами, выстроившимися в проходе в главный зал, располагались плакаты, приглашения, объявления и фотографии, отсылающие к ранним этапам сюрреализма. В главном зале, за оформление которого отвечал Марсель Дюшан 
 
, а за освеще­ние — Ман Рэй 
 
, картины 1920-х годов висели рядом с более ранними работами, что подчеркивало развити
е сюрреалистического «интернаци­она­ла». Зародившись как литературное течение, к концу 1930-х годов сюрреализм уже около 15 лет господствовал в художественном авангарде Парижа. Прежде чем пойти на спад с началом
Второй мировой войны, он стал частью светской культуры Парижа и даже до некотоРой
степени присягнул высокой моде, подобно тому как русский авангард — пусть совсем иначе — присягну 
в свое время революции. Изящество стиля, свойственное сюррелизму, способствоало этому 

­ сближению, которое, в свою очередь, упрочило положение многих представителей направления в обществе. 

Однако поначалу для литераторов и художников-бунтарей, ничуть не стремившихся к социаль­ному успеху, 
была куда более естественной связь с дадаизмом"""

#UNCOMMENT TO UPLOAD============================================================

 
if want_upload:
  uploaded = interactive_upload('Protocol')[0]
# 
_regex_addon = [
    (re.compile(r'[­]'), '-'),
]
TOKENS=tokenize_text( normalize_text(uploaded, replacements_regex+_regex_addon))



class PF(AbstractPatternFactory):
  def __init__(self):
    AbstractPatternFactory.__init__(self, None)
    self._build_ner_patterns()

  def _build_ner_patterns(self):
    def cp(name, tuples):
      return self.create_pattern(name, tuples)
   
    cp('_custom', ( '', search_for, ''))
    

# ---
pf = PF()

av = PM.find_patterns(text_tokens=TOKENS, patterns=pf.patterns)


 
fig = plt.figure(figsize=(20, 6))
ax = plt.axes()
for name in av.vectors:
  ax.plot(av.get_best(name, relu_th=0.8), label='v:'+name, alpha=0.5)
ax.plot([0.5]*av.size, label='0.5', alpha=0.2)
plt.legend(loc='upper right')



GLOBALS__['renderer'].render_color_text(TOKENS, av.get_by_name('_custom'), _range=(0,1) )
# TOKENS

# Fuzzy Matcher

In [0]:
from fuzzy_matcher import AttentionVectors
from ml_tools import sum_probabilities, subtract_probability, relu, momentum_p
import numpy as np

class FuzzyMatcher:
  def __init__(self, av: AttentionVectors):
    self.av = av
    self.constraints = []
    self.incusions = []

  def _add(self, name, max_tokens, to_left):
    assert self.av.has(name), f'there is no vector named {name}'
    self.constraints.append((name, max_tokens, to_left))

  def _include(self, name, include):
    assert self.av.has(name), f'there is no vector named {name}'
    self.incusions.append((name, include))

  def after(self, name, max_tokens: int) -> 'FuzzyMatcher':
    self._add(name, max_tokens, False)
    return self

  def before(self, name, max_tokens: int) -> 'FuzzyMatcher':
    self._add(name, max_tokens, True)
    return self

  def excluding(self, name) -> 'FuzzyMatcher':
    self._include(name, False)
    return self

  def including(self, name) -> 'FuzzyMatcher':
    self._include(name, True)
    return self

  def _get(self, name, p_threshold=0.5):
    _relu = 0.8
    if name[-1] == '*':
      vectors = self.av.get_bests(name[:-1], relu_th=_relu)
      v = sum_probabilities( [relu(v, p_threshold) for v in vectors])
    else:
      v = self.av.get_best(name, relu_th=_relu)

    return v

  def compile(self, p_threshold=0.33, bias=0) -> np.ndarray:
 
    # regions
    _momented = []
    _momented.append(np.zeros(self.av.size))
    for name, max_tokens, to_left in self.constraints:
      v = self._get(name, p_threshold) + bias
      v = relu(v, p_threshold)  # TODO: remove? do not ReLu here?
      momented = momentum_p(v, half_decay=max_tokens, left=to_left)
      momented -= v
#       momented = relu(momented, 0.5)
      _momented.append(momented)

    m_result = sum_probabilities(_momented)

    # incusions
    _included = []
    _included.append(np.zeros(self.av.size))
    for name, include in self.incusions:
      v = self._get(name, p_threshold) + bias
      if include:
        _included.append(relu(v, p_threshold))
    i_result = sum_probabilities(_included)

    # exclusions
    _excluded = []
    _excluded.append(np.zeros(self.av.size))
    for name, include in self.incusions:
      v = self._get(name, p_threshold)+bias
      if not include:
        _excluded.append(relu(v, p_threshold))
    e_result = sum_probabilities(_excluded)

    m_result = sum_probabilities([m_result, i_result])
    m_result = subtract_probability(m_result, e_result)

    #     result = relu(result, 0.001)
    return m_result
  
  
  
FuzzyMatcher2=FuzzyMatcher

### Make patterns

In [0]:
 
from patterns import AbstractPatternFactory, FuzzyPattern


ORG_TYPES = {
  'org_unknown': 'undefined',
  'org_ao':   'Акционерное общество',
  'org_zao':  'Закрытое акционерное общество',
  'org_oao':  'Открытое акционерное общество',
  'org_ooo':  'Общество с ограниченной ответственностью',
  'org_nc':   'Некоммерческая организация',
  'org_fpsi': 'Фонд поддержки социальных инициатив',
  'org_any_1': 'Учреждение',
  'org_any_2': 'Организация'
    
}




class CPF(AbstractPatternFactory):
  def __init__(self):
    AbstractPatternFactory.__init__(self, None)
    self._build_ner_patterns()

  def _build_ner_patterns(self):
    def cp(name, tuples):
      assert tuples[1] != '', name
      return self.create_pattern(name, tuples)

    for o_type in ORG_TYPES.keys():
      if o_type != 'org_unknown':
        cp(o_type, ('', ORG_TYPES[o_type], ' «'))

    
    cp('_named_1', ( ',', 'именуемое в дальнейшем', '«'))
    cp('_named_2', ( ',', 'именуемое', 'далее «'))
#     cp('_named_3', ( ',', 'далее –', '«'))
    
    
    cp('_made_a_deal', ( '', 'заключили', 'настоящий договор о нижеследующем'))
    
    cp('_acting_on', ( ',', 'действующего на основании', ''))
    
    cp('_deal_side_1', ( ',', 'с одной стороны', ''))
    cp('_deal_side_2', ( ',', 'с другой стороны', ''))
    

    cp('_in_face', ( '', 'в лице', ''))
    
    
    px = ', именуемое в дальнейшем'
    cp('_alias_out.1', ( px, 'Покупатель', ''))
    cp('_alias_out.2', ( px, 'Заемщик', ''))
    cp('_alias_out.3', ( px, 'Арендатор', ''))
    cp('_alias_out.4', ( px, 'Жертвователь', ''))
    cp('_alias_out.5', ( px, 'Благотворитель', ''))
    cp('_alias_out.5', ( px, 'Заказчик', ''))
    
    cp('_alias_in.1', ( px, 'Исполнитель', ''))
    cp('_alias_in.2', ( px, 'Продавец', ''))    
    

# ---
contract_pattern_factory = CPF()



In [0]:
for p in contract_pattern_factory.patterns:
  print(p.name, f'\t"{p.prefix_pattern_suffix_tuple[1]}"')

In [0]:
%%time
av = PM.find_patterns(text_tokens=TOKENS, patterns=contract_pattern_factory.patterns)



In [0]:

import numpy as np

from ml_tools import *
from fuzzy_matcher import AttentionVectors
 
_slice=slice(0,400)


#--------------------------------QUOTES attention-
from text_tools import hot_quotes
_quotes_open, _quotes_closing = hot_quotes(TOKENS)
av.add('_quotes_open', _quotes_open, _quotes_open)
av.add('_quotes_closing', _quotes_closing, _quotes_closing)
#----------------------------END OF----QUOTES attention-


# def sharpen(x, blur_size, magnitude=0.5):
#   blurry = smooth_safe(x, blur_size, 'flat') * magnitude
#   x = relu(x - blurry)
#   return x

fm = FuzzyMatcher2(av)

# fm.after('org_*',10)


fm.after('org_*', 20) \
  .after('_deal_side_1', 30) \
  .before('_named_*', 40) \
  .before('_acting_on', 40) \
  .before('_deal_side_*', 100)\
  .before('_made_a_deal', 200) \
  .after('_quotes_open', 6)\
  .before('_quotes_closing', 6)\
  .before('_in_face', 10)

# fm.after('org_*', 20)  .before('_named_*', 40)



vvv = fm.compile(p_threshold=0.33, bias=-0.5)
# vvv =  vvv*vvv*vvv 
# vvv = sharpen(vvv, blur_size=20, magnitude=1)
vvv = relu(vvv, vvv.max() * 0.95)

fig = plt.figure(figsize=(20, 6))
ax = plt.axes()
ax.plot(vvv[_slice], color='red', alpha=0.5)

orig = av.get_by_name('org_ao')
orig = relu(orig, 0.33)
orig = momentum_p(orig, half_decay=10)
ax.plot(orig[_slice] , color='black', alpha=0.5)


GLOBALS__['renderer'].render_color_text(TOKENS[_slice], vvv[_slice], _range=[0,1])

In [0]:

  
#--- test



fm = FuzzyMatcher(av)

fm.after('org_*', 20) .before('_named_*', 40)

vvv = fm.compile(bias = -0.5)
# vvv = sharpen(vvv, blur_size=20, magnitude=1)
vvv = relu(vvv, vvv.max() * 0.9)

fig = plt.figure(figsize=(20, 6))
ax = plt.axes()
ax.plot(vvv[_slice])
ax.plot( [0.5]*len(vvv[_slice]), alpha=0.2 )

_slice=slice(0,200)
GLOBALS__['renderer'].render_color_text(TOKENS[_slice], vvv[_slice])

# Batch

In [0]:
import gspread
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
google_spread = gspread.authorize(GoogleCredentials.get_application_default())

drive.mount('/content/gdrive', force_remount=True)


import glob
def read_documents(filename_prefix):
  texts = {}
  for file in glob.glob(filename_prefix+"*.doc"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.doc FILE!!', file)

  for file in glob.glob(filename_prefix+"*.docx"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.docx FILE!!', file)
      
  return texts

contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
contracts = read_documents(contracts_filename_prefix)

In [0]:
BAD_DOCS="""
МНПЗ 1 Протокол_ГПН-МНПЗ благотвор.doc
Договор хранения нефти_ХранТрансНефть.doc
7. Соглашение БГ Газпромнефть-Региональные продажи.doc
ГПН 3 Протокол заочного СД (пожертвование).docx
Договор 3.docx
МНПЗ 3 Протокол_дарение ДС №135.docx
ЕЮ2 Договор пожертвования (одаряемый).docx
ГПН 3 Договор пожертвования 21.02.2019.doc
ГПН 3 Договор пожертвования 21.02.2019 (1).doc
2. Договор по благ-ти Радуга.docx
Доп.согл3 Доверие.doc
1. Доп. соглашение к Договору_фин.docx
Договор целевого пожертвования.docx
МНГ3 Договор МНГ Пожертвование.docx
"""

BAD_DOCS = [ '/content/gdrive/My Drive/GazpromOil/Contracts/'+x.strip() for x in BAD_DOCS.split('\n') if x.strip()!='' ]
BAD_DOCS 

In [0]:
import numpy as np

# from fuzzy_matcher import FuzzyMatcher
from ml_tools import relu, cut_above
from text_normalize import replacements_regex, normalize_text
from text_tools import hot_quotes, my_punctuation, Tokens, tokenize_text

 


def match_contractor_aliases(_av):
  alias_matcher = FuzzyMatcher2(_av)
  
#   alias_matcher \
#     .after('_named_*', 10).after('_quotes_open', 5) 
  
  alias_matcher \
    .after('_named_*', 10) \
    .including('_alias_*') \
    .before('_in_face', 8) \
    .before('_quotes_closing', 5) \
    .after('_quotes_open', 5) \
    .excluding('hot_punkt') \
    .before('_deal_side_*', 5)

  alias_attention = alias_matcher.compile(p_threshold=0.2, bias=-0.4)
#   alias_attention /= 2.0

#   alias_attention = relu(alias_attention, 0.5)
#   alias_attention = cut_above(alias_attention, 1)
  aliaces_indices = sorted(np.argsort(alias_attention)[::-1][:2])
  return aliaces_indices, alias_attention


fc = 0
export_string = ''
for fn in contracts:
  fc += 1
  print()
  print(fc, '=' * 100)
  print(fc, fn)
  txt = contracts[fn]
  tokens = tokenize_text(normalize_text(txt, replacements_regex))
  tokens = tokens[:300]  # TODO: trimming just for the batch to speed things up

  _av = PM.find_patterns(text_tokens=tokens, patterns=contract_pattern_factory.patterns)

  # --------------------------------QUOTES attention-
  _quotes_open, _quotes_closing = hot_quotes(tokens)
  _av.add('_quotes_open', _quotes_open * 0.5)
  _av.add('_quotes_closing', _quotes_closing * 0.5)

  _av.add('hot_punkt', hot_punkt(tokens))
  # ----------------------------END OF----QUOTES attention-

  _fm = FuzzyMatcher2(_av)

  #   _fm.after('org_*', 20) \
  #     .after('_deal_side_1', 30) \
  #     .after('_quotes_open', 10) \
  #     .before('_quotes_closing', 10) \
  #     .before('_named_*', 40) \
  #     .before('_acting_on', 40) \
  #     .before('_deal_side_*', 100)\
  #     .before('_made_a_deal', 200)\
  #     .before('_in_face', 10)\
  #     .excluding('_alias_*')\

  aliaces_indices, alias_attention = match_contractor_aliases(_av)

  _sel_t = [tokens[aliaces_indices[0]], '---', tokens[aliaces_indices[1]]]
  _sel_v = np.array([alias_attention[aliaces_indices[0]], 0, alias_attention[aliaces_indices[1]]])

  GLOBALS__['renderer'].render_color_text(_sel_t, _sel_v, _range=[0, 1])

  i1 = aliaces_indices[0]
  i2 = aliaces_indices[1]

  export_string += f'{fn.split("/")[-1]}\t{tokens[i1]}\t{tokens[i2]}\t{alias_attention[i1]}\t{alias_attention[i2]}\n'

  alias_attention_sl = slice(i1 - 10, i2 + 10)
  GLOBALS__['renderer'].render_color_text(tokens[alias_attention_sl], alias_attention[alias_attention_sl], _range=(0,1))

#   attention = _fm.compile()
#   attention = relu(attention, 0.5 * np.max(attention))
#   best_indices = np.argsort(alias_attention)[::-1][:10]
#   best_index = np.min(best_indices)
#   last_index = np.max(best_indices)

#   last_index = min(last_index, best_index+100)
# #   print('BI:', best_indices)
#   _slice = slice(best_index-5, last_index)
#   GLOBALS__['renderer'].render_color_text(tokens[_slice], alias_attention[_slice])


print(export_string)
